In [4]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm

# -----------------------------
# 1) تنظیمات اولیه
# -----------------------------

# نام فایل CSV اصلی (خروجی Apify)
INPUT_CSV = "C:/Users/NaMe/OneDrive - Constructor University/Documents/PhD/Tiktok/Tiktok/Data/tiktok.csv"          # اگر اسم فایل تو چیز دیگه‌ایه، همینجا عوضش کن
OUTPUT_CSV = "C:/Users/NaMe/OneDrive - Constructor University/Documents/PhD/Tiktok/Tiktok/Data/tiktok_with_sentiment.csv"

# حداکثر طول متن که به مدل می‌دیم (برای اینکه خیلی بلند نباشه)
MAX_TEXT_LEN = 256

# -----------------------------
# 2) خواندن فایل CSV
# -----------------------------
print("📥 در حال خواندن فایل CSV ...")
df = pd.read_csv(INPUT_CSV)

# چک کنیم ستون 'text' وجود داشته باشه
if "text" not in df.columns:
    raise ValueError("ستون 'text' در فایل CSV پیدا نشد. لطفاً مطمئن شو نام ستون دقیقاً 'text' است.")

print(f"✅ تعداد ردیف‌ها: {len(df)}")

# -----------------------------
# 3) بارگذاری مدل‌ها
# -----------------------------
print("🤖 در حال بارگذاری مدل‌های Sentiment و Emotion (بار اول کمی طول می‌کشه)...")

# مدل احساس مثبت/منفی/خنثی
sentiment_model = pipeline(
    task="sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment-latest"
)

# مدل Emotion (joy, anger, sadness, fear, ...)
emotion_model = pipeline(
    task="text-classification",
    model="j-hartmann/emotion-english-distilroberta-base",
    return_all_scores=False
)

# -----------------------------
# 4) توابع کمکی
# -----------------------------
def clean_text(text):
    """تبدیل به استرینگ، حذف فاصله‌های اضافه، کوتاه کردن متن طولانی"""
    if not isinstance(text, str):
        return ""
    text = text.strip()
    if len(text) > MAX_TEXT_LEN:
        text = text[:MAX_TEXT_LEN]
    return text

def analyze_sentiment(text):
    text = clean_text(text)
    if text == "":
        return None, None
    try:
        result = sentiment_model(text)[0]
        label = result["label"]      # مثلا: "Positive" / "Negative" / "Neutral"
        score = float(result["score"])
        return label, score
    except Exception as e:
        print("خطا در sentiment:", e)
        return None, None

def analyze_emotion(text):
    text = clean_text(text)
    if text == "":
        return None, None
    try:
        result = emotion_model(text)[0]
        label = result["label"]      # مثلا: "joy", "anger", "sadness", ...
        score = float(result["score"])
        return label, score
    except Exception as e:
        print("خطا در emotion:", e)
        return None, None

# -----------------------------
# 5) اجرای تحلیل روی همه کپشن‌ها
# -----------------------------
sentiment_labels = []
sentiment_scores = []
emotion_labels = []
emotion_scores = []

print("📊 در حال تحلیل کپشن‌ها ...")

for text in tqdm(df["text"], total=len(df)):
    s_label, s_score = analyze_sentiment(text)
    e_label, e_score = analyze_emotion(text)

    sentiment_labels.append(s_label)
    sentiment_scores.append(s_score)
    emotion_labels.append(e_label)
    emotion_scores.append(e_score)

# اضافه کردن ستون‌های جدید
df["sentiment_label"] = sentiment_labels
df["sentiment_score"] = sentiment_scores
df["emotion_label"] = emotion_labels
df["emotion_score"] = emotion_scores

# -----------------------------
# 6) ذخیره نتیجه
# -----------------------------
print(f"💾 در حال ذخیره فایل جدید: {OUTPUT_CSV}")
df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
print("✅ تمام شد! حالا می‌تونی این فایل را در Excel / SPSS / Python تحلیل کنی.")


📥 در حال خواندن فایل CSV ...
✅ تعداد ردیف‌ها: 905
🤖 در حال بارگذاری مدل‌های Sentiment و Emotion (بار اول کمی طول می‌کشه)...


c:\Users\NaMe\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\NaMe\.cache\huggingface\hub\models--cardiffnlp--twitter-roberta-base-sentiment-latest. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is n

📊 در حال تحلیل کپشن‌ها ...


100%|██████████| 905/905 [01:59<00:00,  7.57it/s]

💾 در حال ذخیره فایل جدید: C:/Users/NaMe/OneDrive - Constructor University/Documents/PhD/Tiktok/Tiktok/Data/tiktok_with_sentiment.csv
✅ تمام شد! حالا می‌تونی این فایل را در Excel / SPSS / Python تحلیل کنی.


In [6]:
import pandas as pd

df = pd.read_csv("C:/Users/NaMe/OneDrive - Constructor University/Documents/PhD/Tiktok/Tiktok/Data/tiktok_with_sentiment.csv")

# میانگین شدت احساسات (اسکور)
mean_sentiment = df["sentiment_score"].mean()
mean_emotion = df["emotion_score"].mean()

print("میانگین شدت احساس:", mean_sentiment)
print("میانگین شدت Emotion:", mean_emotion)

# توزیع برچسب‌ها (مثلاً چندتا مثبت؟ چندتا منفی؟)
print(df["sentiment_label"].value_counts())
print(df["emotion_label"].value_counts())


میانگین شدت احساس: 0.7419687488320407
میانگین شدت Emotion: 0.6719238547912827
sentiment_label
neutral     408
negative    355
positive    136
Name: count, dtype: int64
emotion_label
fear        424
joy         190
anger       119
neutral     105
sadness      27
disgust      24
surprise     10
Name: count, dtype: int64


In [7]:
corr = df[["sentiment_score", "emotion_score", 
           "diggCount", "commentCount", "shareCount", "playCount"]].corr()

print(corr)


                 sentiment_score  emotion_score  diggCount  commentCount  \
sentiment_score         1.000000       0.014374   0.019371      0.027320   
emotion_score           0.014374       1.000000   0.072844      0.064847   
diggCount               0.019371       0.072844   1.000000      0.883837   
commentCount            0.027320       0.064847   0.883837      1.000000   
shareCount              0.024788       0.081119   0.711433      0.621452   
playCount               0.038684       0.070256   0.754814      0.582157   

                 shareCount  playCount  
sentiment_score    0.024788   0.038684  
emotion_score      0.081119   0.070256  
diggCount          0.711433   0.754814  
commentCount       0.621452   0.582157  
shareCount         1.000000   0.692124  
playCount          0.692124   1.000000  


In [11]:
df["engagement"] = df["diggCount"] + df["commentCount"] + df["shareCount"] + df["playCount"]


In [13]:
import pandas as pd
df = pd.read_csv("C:/Users/NaMe/OneDrive - Constructor University/Documents/PhD/Tiktok/Tiktok/Data/tiktok_with_sentiment.csv")


In [14]:
df = df.dropna(subset=["sentiment_score"])


In [15]:
df["diggCount"] = pd.to_numeric(df["diggCount"], errors="coerce")
df["commentCount"] = pd.to_numeric(df["commentCount"], errors="coerce")
df["shareCount"] = pd.to_numeric(df["shareCount"], errors="coerce")
df["playCount"] = pd.to_numeric(df["playCount"], errors="coerce")


In [17]:
import statsmodels.api as sm


In [18]:
import pandas as pd


# 1) خواندن فایل
df = pd.read_csv("C:/Users/NaMe/OneDrive - Constructor University/Documents/PhD/Tiktok/Tiktok/Data/tiktok_with_sentiment.csv")

# 2) تبدیل ستون‌های عددی
for col in ["diggCount", "commentCount", "shareCount", "playCount"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# 3) حذف ردیف‌هایی که احساساتشان NA است
df = df.dropna(subset=["sentiment_score"])

# 4) ساخت متغیر engagement
df["engagement"] = df["diggCount"] + df["commentCount"] + df["shareCount"] + df["playCount"]

# 5) مدل رگرسیون
X = df["sentiment_score"]
y = df["engagement"]

X = sm.add_constant(X)   # اضافه‌کردن b0

model = sm.OLS(y, X).fit()

print(model.summary())


                            OLS Regression Results                            
Dep. Variable:             engagement   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.289
Date:                Sun, 23 Nov 2025   Prob (F-statistic):              0.257
Time:                        12:36:37   Log-Likelihood:                -14972.
No. Observations:                 899   AIC:                         2.995e+04
Df Residuals:                     897   BIC:                         2.996e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const             1.36e+05   7.28e+05     

In [ ]:
import statsmodels.api as sm
import pandas as pd

df = pd.read_csv("tiktok_with_sentiment.csv")

# تبدیل به numeric
for col in ["diggCount", "commentCount", "shareCount", "playCount", "videoMeta.duration"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# تعداد هشتگ‌ها
df["hashtag_count"] = df["text"].str.count("#")

# ساخت engagement
df["engagement"] = df["diggCount"] + df["commentCount"] + df["shareCount"] + df["playCount"]

# مدل چندمتغیره
X = df[[
    "sentiment_score",
    "emotion_score",
    "hashtag_count",
    "videoMeta.duration"
]]

y = df["engagement"]

X = sm.add_constant(X)
model = sm.OLS(y, X).fit()

print(model.summary())
